In [ ]:
# initialise the qgis application
# TODO - please add the path to your qgis installation and then run this script

In [ ]:
# import statements
import processing
import os
from osgeo import gdal, osr
import numpy as np

In [ ]:
# Define the input/output paths
input_dem = ''   # Depressionless DEM file
output_dir = ''  # Output directory path to store the temporary and final outputs

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Instead of computing the accumulation i.e. no of incoming pixels flowing into it
We can also use this directly from the side-output when we computed the stream raster

In [ ]:
# Define temp path for catchment raster
accumulation_path = os.path.join(output_dir, 'accumulation_raster.tif')

# Run the processing command
processing.run("grass:r.watershed", {
    'elevation': input_dem,
    'depression': None,
    'flow': None,
    'disturbed_land': None,
    'blocking': None,
    'threshold': 100000,
    'max_slope_length': None,
    'convergence': 5,
    'memory': 100000,
    '-s': False,
    '-m': False,
    '-4': False,
    '-a': False,
    '-b': False,
    'accumulation': accumulation_path,
    # Map other outputs if required like drainage or stream ...
    # 'basin': 'TEMPORARY_OUTPUT',
    # 'drainage': 'TEMPORARY_OUTPUT',
    # 'stream': 'TEMPORARY_OUTPUT',
    'GRASS_REGION_PARAMETER': None,
    'GRASS_REGION_CELLSIZE_PARAMETER': 0,
    'GRASS_RASTER_FORMAT_OPT': '',
    'GRASS_RASTER_FORMAT_META': ''
})

In [ ]:
# Open the accumulation raster
accumulation_ds = gdal.Open(accumulation_path)
accumulation_band = accumulation_ds.GetRasterBand(1)
accumulation_data = accumulation_band.ReadAsArray()

# Multiply by 0.09 to convert to hectares
hectares_data = accumulation_data * 0.09

# Define the output path for the updated raster
hectares_path = os.path.join(output_dir, 'catchment_area_ha.tif')

# Create the output raster
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(hectares_path, accumulation_ds.RasterXSize, accumulation_ds.RasterYSize, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(accumulation_ds.GetGeoTransform())
out_ds.SetProjection(accumulation_ds.GetProjection())

# Write the updated data to the output raster
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(hectares_data)
out_band.SetNoDataValue(-9999)

# Flush data to disk and close datasets
out_band.FlushCache()
out_ds = None
accumulation_ds = None